In [19]:
! dlt --version

dlt 1.6.1


In [12]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

In [13]:
@dlt.resource(name="rides",write_disposition="append")
def nyc_taxi_rides():
    """
    This source retrieves NYC taxi rides from the NYC Taxi & Limousine Commission's API.
    """
    client = RESTClient(
        base_url="https://us-central1-dlthub-analytics.cloudfunctions.net/",
        paginator=PageNumberPaginator(base_page=1, total_path=None),
    )

    for page in client.paginate("data_engineering_zoomcamp_api"):
        yield page

In [14]:
# Define the DLTHub pipeline
pipeline = dlt.pipeline(
    pipeline_name="nyc_taxi",
    destination="bigquery",
    dataset_name="nyc_taxi_dataset",
)

In [15]:
# Ingest data into the pipeline
pipeline.run(nyc_taxi_rides)

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x70b9cb8e2510>, metrics={'1739768351.6999278': [{'started_at': DateTime(2025, 2, 17, 4, 59, 39, 686621, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2025, 2, 17, 4, 59, 58, 361322, tzinfo=Timezone('UTC')), 'job_metrics': {'_dlt_pipeline_state.436a2ceb5b.jsonl': LoadJobMetrics(job_id='_dlt_pipeline_state.436a2ceb5b.jsonl', file_path='/var/dlt/pipelines/nyc_taxi/load/normalized/1739768351.6999278/started_jobs/_dlt_pipeline_state.436a2ceb5b.0.jsonl', table_name='_dlt_pipeline_state', started_at=DateTime(2025, 2, 17, 4, 59, 50, 788883, tzinfo=Timezone('UTC')), finished_at=DateTime(2025, 2, 17, 4, 59, 54, 622783, tzinfo=Timezone('UTC')), state='completed', remote_url=None), 'rides.af77b7fd80.jsonl': LoadJobMetrics(job_id='rides.af77b7fd80.jsonl', file_path='/var/dlt/pipelines/nyc_taxi/load/normalized/1739768351.6999278/started_jobs/rides.af77b7fd80.0.jsonl', table_name='rides', started_at=DateTime(2025, 2, 17, 4, 59, 50, 83781

In [ ]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff(trip_dropoff_date_time, trip_pickup_date_time,MINUTE))
            FROM rides;
            """
        )
    # Prints column values of the first row
    print(res)

[Row((-12.298199999999992,), {'f0_': 0})]
